In [1]:
# installing libraries that let pandas read parquet files

%pip install -U pyarrow fastparquet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# importing the Path class so you can create Path objects directly
from pathlib import Path

In [3]:
#importing pandas and reading the parquet file

import pandas as pd
df = pd.read_parquet("si_revwar.parquet")

In [4]:
# checking table to confirm it loaded correctly
display(df)

,collectionsURL,unitCode,EDANurl,title,creditLine,date,identifier,name,notes,objectType,...,publisher,topic,indexed_dates,indexed_names,indexed_object_types,indexed_places,indexed_topics,mediaCount,mediaURLs,thumbnail
0,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_XX108A,George Washington,"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""1803?""}","{""Object number"": ""XX108A""}","{""Attributed to"": ""William Winstanley, born En...",None,"{""Type"": ""Painting""}",...,None,"{""Topic"": ""Occupation\\political\\president\nP...","[""1800s""]","[""Washington, George"", ""Winstanley, William""]","[""Paintings""]",None,"[""Occupations"", ""Men"", ""Portraits"", ""Political...",1,"[""https://ids.si.edu/ids/deliveryService?id=SA...",https://ids.si.edu/ids/iiif/SAAM-XX108A_1/full...
1,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2024.40.7,Untitled (Quilt Sample Section),"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""ca. 1800""}","{""Object number"": ""2024.40.7""}","{""Artist"": ""Unidentified""}",None,"{""Type"": ""Decorative Arts-Fiber\nFolk Art\nQui...",...,None,None,"[""1800s"", ""1790s""]",None,"[""Quilts"", ""Decorative arts"", ""Folk art""]",None,None,0,None,None
2,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2024.24,Portrait of Thayendanegea (Joseph Brant),"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""after 1797""}","{""Object number"": ""2024.24""}","{""Artist"": ""Unidentified"", ""Copy after"": ""Char...",None,"{""Type"": ""Painting""}",...,None,"{""Topic"": ""Portrait male""}","[""1790s""]","[""Peale, Charles Willson"", ""Portrait male""]","[""Paintings""]",None,"[""Men"", ""Portraits""]",0,None,None
3,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2019.6.12,The Davis Children (Eliza Cheever Davis and Jo...,"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""1795""}","{""Object number"": ""2019.6.12""}","{""Artist"": ""Edward Savage, born Princeton, MA ...",None,"{""Type"": ""Painting""}",...,None,"{""Topic"": ""Landscape\nObject\\flower\\lily\nPo...","[""1790s""]","[""Savage, Edward"", ""Davis, Eliza Cheever"", ""Da...","[""Paintings""]",None,"[""Women"", ""Landscapes"", ""Men"", ""Portraits"", ""F...",1,"[""https://ids.si.edu/ids/deliveryService?id=SA...",https://ids.si.edu/ids/iiif/SAAM-2019.6.12_1/f...
4,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2017.23.2,Elizabeth Waters (Mrs. Richard) Cromwell,"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""ca. 1791-1794""}","{""Object number"": ""2017.23.2""}","{""Artist"": ""Charles Peale Polk, born Annapolis...",None,"{""Type"": ""Painting""}",...,None,None,"[""1780s"", ""1790s""]","[""Polk, Charles Peale""]","[""Paintings""]",None,None,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12662,https://collections.si.edu/search/detail/edanm...,NMAH,edanmdm:nmah_1000617,"2 Dollars, Maryland, 1775","{""Credit Line"": ""The Heirs of William and Denn...","{""date made"": ""1775"", ""date on object"": ""1775-...","{""ID Number"": ""NU.220760.018"", ""accession numb...","{""maker"": ""Green, Frederick""}","{""Location"": ""Currently not on view""}","{""Object Name"": ""currency\nnote""}",...,None,None,"[""1770s""]","[""Green, Frederick""]","[""Money"", ""note""]","[""United States"", ""Maryland""]","[""NNC Colonial Currency""]",0,None,None
12663,https://collections.si.edu/search/detail/edanm...,NMAH,edanmdm:nmah_1000616,"2 2/3 Dollars, Maryland, 1775","{""Credit Line"": ""The Heirs of William and Denn...","{""date made"": ""1775"", ""date on object"": ""1775-...","{""ID Number"": ""NU.220760.017"", ""accession numb...","{""maker"": ""Green, Frederick""}","{""Location"": ""Currently not on view""}","{""Object Name"": ""currency\nnote""}",...,None,None,"[""1770s""]","[""Green, Frederick""]","[""Money"", ""note""]","[""United S

In [5]:
# checking what Python types are in the column indexed_object_types
df["indexed_object_types"].map(type).value_counts()


indexed_object_types
<class 'str'>         12663
<class 'NoneType'>        4
Name: count, dtype: int64

In [6]:
# there are only NoneType values in the column, so I will drop those rows
df = df[df["indexed_object_types"].map(type) != type(None)]


In [7]:
# the rest of the values are strings that look like JSON lists
# I want to parse them from strings to actual JSON lists
# (for example, "["note","voucher"]"" should become just ["note", "voucher"])
import json

def parse_indexed_object_types(cell):
    """
    converts a string that looks like a JSON list into a real JSON list
    """
    return json.loads(cell)          # parses original string into a JSON list

# creating new column with the parsed object types
df["object_types_list"] = df["indexed_object_types"].apply(parse_indexed_object_types)

In [8]:
# checking what Python types are in the new column object_types_list now (they should all be lists)
df["object_types_list"].map(type).value_counts()

object_types_list
<class 'list'>    12663
Name: count, dtype: int64

In [9]:
# normalizing the lists in object_types_list

import re # for regular expressions
import unicodedata # for handling Unicode characters consistently

def normalize_token_basic(t):
    s = unicodedata.normalize("NFKC", str(t)) #transforms the string into a standardized Unicode form:
    s = s.lower().strip() #turns everything to lowercase and removes leading/trailing whitespace
    s = re.sub(r"[\u2010-\u2015\-_\/]+", " ", s) #replaces various dashes, underscores, and slashes with a single space
    return s #returns the cleaned-up string
df["object_types_norm"] = df["object_types_list"].apply(
    lambda items: [normalize_token_basic(x) for x in items]
)   # loops through each row in the column and passes the value in that row to the normalization function, applying it to each item in the lists


In [10]:
# count unique full-cell values in column object_types_norm

# 1. converting each list into a string so pandas can compare easily
df["object_types_str"] = df["object_types_norm"].apply(str)

# 2. counting unique values and their frequencies
counts_df = (
    df["object_types_str"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "object_types", "object_types_str": "count"})
)

# 3. inspecting top values
display(counts_df.head(50))



,count,count
0,['note'],2470
1,['silhouettes'],1845
2,['prints'],1422
3,['coins (money)'],1075
4,['paintings'],297
5,"['spoons', 'flatware']",191
6,['medals'],170
7,"['bottles', 'bottle; pharmaceutical container']",145
8,['decorative arts'],114
9,"['jars', 'jar; pharmaceutical container']",111


In [11]:
# there are a lot of variations of the same object types
# exporting the CSV to review the different object types and define a grouping strategy using an LLM
# canonicalization criteria provided to the LLM:
# 1. normalize spelling
# 2. merge synonyms under a single name (e.g., "pipe tampers", "pipe, tobaccos" → "pipes")
# 2. remove overly specific qualifiers (e.g., "silver spoon", "table spoon" → "spoons")
# 3. collapse minor variants into parent category (e.g., "dress, 1 piece", "dress fragments" → "dresses")
# 4. keep distinct when functionally differs (e.g., "box, stamps"  ≠ "box, doughs")
# 5. use plural form consistently (e.g., "painting" → "paintings")

counts_df.to_csv("unique_object_types_fullcells.csv")

In [12]:
# loading and displaying the curated canonicalization mapping

mapping_df = pd.read_csv("canonical_map.csv")

# this CSV has two key columns:
# - "original_fullcell": the original unique string from the database
# - "canonical_plural": the standardized category, in plural form

display(mapping_df)

,original_fullcell,frequency,tokens_parsed,chosen_main_token,canonical_plural
0,"['fans (costume accessories)', 'fan; accessori...",6,fans; fan; accessories carried; female; access...,accessories carried,accessories carrieds
1,"['bag; bags; accessories carried; female', 'ba...",5,bag; bags; accessories carried; female; bags; ...,accessories carried,accessories carrieds
2,"['bags', 'bags (containers)', 'accessories car...",1,bags; bags; accessories carried; pocket book,accessories carried,accessories carrieds
3,"['bag; bags; accessories carried; female', 'ba...",1,bag; bags; accessories carried; female; bags; ...,accessories carried,accessories carrieds
4,"['fan, fragment', 'fans (costume accessories)'...",1,"fan, fragment; fans; accessories carried; fan,...",accessories carried,accessories carrieds
...,...,...,...,...,...
1731,['worm'],1,worm,worm,worms
1732,"['wrench, barrel', 'mold, shot', 'mold, bullet...",1,"wrench, barrel; mold, shot; mold, bullet; 41cc...","wrench, barrel","wrench, barrels"
1733,"['wrought iron bridge spike', 'wrought iron br...",1,wrought iron bridge spike; wrought iron bridge...,wrought iron bridge spike,wrought iron bridge spikes
1734,"['yardage', 'yardage']",1,yardage; yardage,yardage,yardages


In [13]:
# turning that csv into a dictionary

mapping_dict = pd.Series(mapping_df.canonical_plural.values, index=mapping_df.original_fullcell).to_dict()


In [14]:
# applying the dictionary to the dataframe
# first, I need the column in the same format as "original_fullcell", which means
# converting the lists back to strings

df["object_types_str"] = df["object_types_norm"].astype(str)

In [15]:
# mapping each row to its canonical category using the dictionary
# this function goes through every element in the column object_types_str from df, 
# looks them up in the mapping_dict dictionary and creates the new column canonical_type corresponding element form the  column from 

df["canonical_type"] = df["object_types_str"].map(mapping_dict)

In [16]:
# checking how many rows successfully mapped

print("rows mapped:", df["canonical_type"].notna().sum())
print("rows unmapped:", df["canonical_type"].isna().sum())

rows mapped: 12663
rows unmapped: 0


In [17]:
# displaying the df to see if the column was added correctly
display(df)

,collectionsURL,unitCode,EDANurl,title,creditLine,date,identifier,name,notes,objectType,...,indexed_object_types,indexed_places,indexed_topics,mediaCount,mediaURLs,thumbnail,object_types_list,object_types_norm,object_types_str,canonical_type
0,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_XX108A,George Washington,"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""1803?""}","{""Object number"": ""XX108A""}","{""Attributed to"": ""William Winstanley, born En...",None,"{""Type"": ""Painting""}",...,"[""Paintings""]",None,"[""Occupations"", ""Men"", ""Portraits"", ""Political...",1,"[""https://ids.si.edu/ids/deliveryService?id=SA...",https://ids.si.edu/ids/iiif/SAAM-XX108A_1/full...,[Paintings],[paintings],['paintings'],paintings
1,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2024.40.7,Untitled (Quilt Sample Section),"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""ca. 1800""}","{""Object number"": ""2024.40.7""}","{""Artist"": ""Unidentified""}",None,"{""Type"": ""Decorative Arts-Fiber\nFolk Art\nQui...",...,"[""Quilts"", ""Decorative arts"", ""Folk art""]",None,None,0,None,None,"[Quilts, Decorative arts, Folk art]","[quilts, decorative arts, folk art]","['quilts', 'decorative arts', 'folk art']",decorative arts
2,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2024.24,Portrait of Thayendanegea (Joseph Brant),"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""after 1797""}","{""Object number"": ""2024.24""}","{""Artist"": ""Unidentified"", ""Copy after"": ""Char...",None,"{""Type"": ""Painting""}",...,"[""Paintings""]",None,"[""Men"", ""Portraits""]",0,None,None,[Paintings],[paintings],['paintings'],paintings
3,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2019.6.12,The Davis Children (Eliza Cheever Davis and Jo...,"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""1795""}","{""Object number"": ""2019.6.12""}","{""Artist"": ""Edward Savage, born Princeton, MA ...",None,"{""Type"": ""Painting""}",...,"[""Paintings""]",None,"[""Women"", ""Landscapes"", ""Men"", ""Portraits"", ""F...",1,"[""https://ids.si.edu/ids/deliveryService?id=SA...",https://ids.si.edu/ids/iiif/SAAM-2019.6.12_1/f...,[Paintings],[paintings],['paintings'],paintings
4,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2017.23.2,Elizabeth Waters (Mrs. Richard) Cromwell,"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""ca. 1791-1794""}","{""Object number"": ""2017.23.2""}","{""Artist"": ""Charles Peale Polk, born Annapolis...",None,"{""Type"": ""Painting""}",...,"[""Paintings""]",None,None,0,None,None,[Paintings],[paintings],['paintings'],paintings
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12662,https://collections.si.edu/search/detail/edanm...,NMAH,edanmdm:nmah_1000617,"2 Dollars, Maryland, 1775","{""Credit Line"": ""The Heirs of William and Denn...","{""date made"": ""1775"", ""date on object"": ""1775-...","{""ID Number"": ""NU.220760.018"", ""accession numb...","{""maker"": ""Green, Frederick""}","{""Location"": ""Currently not on view""}","{""Object Name"": ""currency\nnote""}",...,"[""Money"", ""note""]","[""United States"", ""Maryland""]","[""NNC Colonial Currency""]",0,None,None,"[Money, note]","[money, note]","['money', 'note']",bank notes
12663,https://collections.si.edu/search/detail/edanm...,NMAH,edanmdm:nmah_1000616,"2 2/3 Dollars, Maryland, 1775","{""Credit Line"": ""The Heirs of William and Denn...","{""date made"": ""1775"", ""date on object"": ""1775-...","{""ID Number"": ""NU.220760.017"", ""accession numb...","{""maker"": ""Green, Frederick""}","{""Location"": ""Currently not on view""}","{""Object Name"": ""currency\nnote""}",...,"[""Money"", ""note""]","[""United States"", ""Maryland""]","[""NNC Colonial Currency""]",0,None,None,"[Money, note]","[money, note]","['money', 'note']",bank notes
12664,https://co

In [18]:
# checking the first 50 rows with original and canonical categories side by side

df[["object_types_str", "canonical_type"]].head(50)

,object_types_str,canonical_type
0,['paintings'],paintings
1,"['quilts', 'decorative arts', 'folk art']",decorative arts
2,['paintings'],paintings
3,['paintings'],paintings
4,['paintings'],paintings
5,['paintings'],paintings
6,"['quilts', 'decorative arts', 'folk art']",decorative arts
7,"['quilts', 'decorative arts', 'folk art']",decorative arts
8,['decorative arts'],decorative arts
9,['sculpture'],sculptures


In [19]:
# checking the frequency of the top 40 canonical type created

df["canonical_type"].value_counts().head(40)

canonical_type
bank notes            2796
prints                1884
silhouettes           1852
coins                 1122
paintings              346
jars                   286
medals                 247
flatwares              221
correspondences        204
bottles                183
decorative arts        127
dresses                105
books                   84
posters                 61
badges                  59
chairs                  59
pipes                   58
plates                  58
pistols                 56
uncut sheets            53
newspapers              52
drawings                51
quilts                  48
pianos                  43
muskets                 42
samplers                40
fire marks              40
cups                    34
checks                  33
fashion plates          33
documents               33
andirons                32
laces                   32
measures                28
archival materials      27
shoes                   27
stamp, butter

In [20]:
# exporting unique canonical categories with counts

# 1. counting frequency of each canonical category
counts = (
    df["canonical_type"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "canonical_type", "canonical_type": "count"})
)

# 2. inspecting the top categories
print(counts.head(20))

# 3. exporting to CSV for review, to define key verb families with a mixed strategy: 
# overall grouping using self-defined action families prompted to an LLM, followed by manual review and correction

counts.to_csv("canonical_types_counts.csv", index=False)

# proposed action families:
# 1. Commemorate & symbolize 
# 2. Decorate & furnish Dress & accessorize 
# 3. Eat, cook & drink 
# 4. Fight 
# 5. Heal & care 
# 6. Ignite & manage fire 
# 7. Measure & navigate 
# 8. Perform music 
# 9. Play 
# 10. Read, write & record 
# 11. Smoke 
# 12. Textile making 
# 13. Work & build 
# 14. Worship 
# 15. Other

              count  count
0        bank notes   2796
1            prints   1884
2       silhouettes   1852
3             coins   1122
4         paintings    346
5              jars    286
6            medals    247
7         flatwares    221
8   correspondences    204
9           bottles    183
10  decorative arts    127
11          dresses    105
12            books     84
13          posters     61
14           badges     59
15           chairs     59
16            pipes     58
17           plates     58
18          pistols     56
19     uncut sheets     53


In [21]:
# loading and displaying the curated object - verb family mapping
# it has two key columns: canonical_type and action_family
map_df = pd.read_csv("object_verb_mapping.csv")
display(map_df)

,canonical_type,count_by_canontype,action_family,multi_action
0,pistols,56,Fight,False
1,muskets,42,Fight,False
2,swords,17,Fight,False
3,bows,12,Fight,False
4,rifles,10,Fight,False
...,...,...,...,...
859,set squares,1,Work & build,False
860,bolts,1,Work & build,False
861,crosses,5,Worship,False
862,"cross, roundeds",2,Worship,False


In [22]:
# creating df_with_action by taking the original df (that already has the canonical_type column) 
# and merging it with map_df, based on the canonical_type

df_with_action = df.merge(
    map_df,
    how="left"
)

# displaying df_with_actions
display(df_with_action)

,collectionsURL,unitCode,EDANurl,title,creditLine,date,identifier,name,notes,objectType,...,mediaCount,mediaURLs,thumbnail,object_types_list,object_types_norm,object_types_str,canonical_type,count_by_canontype,action_family,multi_action
0,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_XX108A,George Washington,"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""1803?""}","{""Object number"": ""XX108A""}","{""Attributed to"": ""William Winstanley, born En...",None,"{""Type"": ""Painting""}",...,1,"[""https://ids.si.edu/ids/deliveryService?id=SA...",https://ids.si.edu/ids/iiif/SAAM-XX108A_1/full...,[Paintings],[paintings],['paintings'],paintings,346.0,"Portray, display & decorate",False
1,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2024.40.7,Untitled (Quilt Sample Section),"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""ca. 1800""}","{""Object number"": ""2024.40.7""}","{""Artist"": ""Unidentified""}",None,"{""Type"": ""Decorative Arts-Fiber\nFolk Art\nQui...",...,0,None,None,"[Quilts, Decorative arts, Folk art]","[quilts, decorative arts, folk art]","['quilts', 'decorative arts', 'folk art']",decorative arts,127.0,"Portray, display & decorate",False
2,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2024.24,Portrait of Thayendanegea (Joseph Brant),"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""after 1797""}","{""Object number"": ""2024.24""}","{""Artist"": ""Unidentified"", ""Copy after"": ""Char...",None,"{""Type"": ""Painting""}",...,0,None,None,[Paintings],[paintings],['paintings'],paintings,346.0,"Portray, display & decorate",False
3,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2019.6.12,The Davis Children (Eliza Cheever Davis and Jo...,"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""1795""}","{""Object number"": ""2019.6.12""}","{""Artist"": ""Edward Savage, born Princeton, MA ...",None,"{""Type"": ""Painting""}",...,1,"[""https://ids.si.edu/ids/deliveryService?id=SA...",https://ids.si.edu/ids/iiif/SAAM-2019.6.12_1/f...,[Paintings],[paintings],['paintings'],paintings,346.0,"Portray, display & decorate",False
4,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2017.23.2,Elizabeth Waters (Mrs. Richard) Cromwell,"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""ca. 1791-1794""}","{""Object number"": ""2017.23.2""}","{""Artist"": ""Charles Peale Polk, born Annapolis...",None,"{""Type"": ""Painting""}",...,0,None,None,[Paintings],[paintings],['paintings'],paintings,346.0,"Portray, display & decorate",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12658,https://collections.si.edu/search/detail/edanm...,NMAH,edanmdm:nmah_1000617,"2 Dollars, Maryland, 1775","{""Credit Line"": ""The Heirs of William and Denn...","{""date made"": ""1775"", ""date on object"": ""1775-...","{""ID Number"": ""NU.220760.018"", ""accession numb...","{""maker"": ""Green, Frederick""}","{""Location"": ""Currently not on view""}","{""Object Name"": ""currency\nnote""}",...,0,None,None,"[Money, note]","[money, note]","['money', 'note']",bank notes,14.0,Pay & exchange,False
12659,https://collections.si.edu/search/detail/edanm...,NMAH,edanmdm:nmah_1000616,"2 2/3 Dollars, Maryland, 1775","{""Credit Line"": ""The Heirs of William and Denn...","{""date made"": ""1775"", ""date on object"": ""1775-...","{""ID Number"": ""NU.220760.017"", ""accession numb...","{""maker"": ""Green, Frederick""}","{""Location"": ""Currently not on view""}","{""Object Name"": ""currency\nnote""}",...,0,None,None,"[Money, note]","[money, note]","['money', 'note']",bank notes,14.0,Pay & exchange,False
12660,https://collections.si.edu/search/detail/edanm...,NMAH,edanmdm:nmah_1000614,Washington City Canal Lottery Ticket,"{""Credit Line"": ""Gift of The Heirs of William ...","{""date made"": ""ca 1796""}","{""ID Number"": ""DL.220760.016"", ""

In [23]:
# checking if all elements have been assigned an action
print(df_with_action['action_family'].isnull().sum())


1


In [24]:
df_with_action.to_csv("final_mapped_database.csv")
display(df_with_action)
# by reviewing the data, I notice that the action family defined as "Contain, store & preserve" contains mostly jars
# however, by looking at more details in other columns it is clear that these jars where used as pharmaceutical containers
# I re-tag the objects inside this action family to better match their purpose ("Heal & care")
# I also manually assign action families to other objects that were tagged as "Other".
# Importing the curated file with these changes and other manual re-tagging edits
df_with_action = pd.read_csv("final_mapped_database_manual.csv")


,collectionsURL,unitCode,EDANurl,title,creditLine,date,identifier,name,notes,objectType,...,mediaCount,mediaURLs,thumbnail,object_types_list,object_types_norm,object_types_str,canonical_type,count_by_canontype,action_family,multi_action
0,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_XX108A,George Washington,"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""1803?""}","{""Object number"": ""XX108A""}","{""Attributed to"": ""William Winstanley, born En...",None,"{""Type"": ""Painting""}",...,1,"[""https://ids.si.edu/ids/deliveryService?id=SA...",https://ids.si.edu/ids/iiif/SAAM-XX108A_1/full...,[Paintings],[paintings],['paintings'],paintings,346.0,"Portray, display & decorate",False
1,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2024.40.7,Untitled (Quilt Sample Section),"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""ca. 1800""}","{""Object number"": ""2024.40.7""}","{""Artist"": ""Unidentified""}",None,"{""Type"": ""Decorative Arts-Fiber\nFolk Art\nQui...",...,0,None,None,"[Quilts, Decorative arts, Folk art]","[quilts, decorative arts, folk art]","['quilts', 'decorative arts', 'folk art']",decorative arts,127.0,"Portray, display & decorate",False
2,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2024.24,Portrait of Thayendanegea (Joseph Brant),"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""after 1797""}","{""Object number"": ""2024.24""}","{""Artist"": ""Unidentified"", ""Copy after"": ""Char...",None,"{""Type"": ""Painting""}",...,0,None,None,[Paintings],[paintings],['paintings'],paintings,346.0,"Portray, display & decorate",False
3,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2019.6.12,The Davis Children (Eliza Cheever Davis and Jo...,"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""1795""}","{""Object number"": ""2019.6.12""}","{""Artist"": ""Edward Savage, born Princeton, MA ...",None,"{""Type"": ""Painting""}",...,1,"[""https://ids.si.edu/ids/deliveryService?id=SA...",https://ids.si.edu/ids/iiif/SAAM-2019.6.12_1/f...,[Paintings],[paintings],['paintings'],paintings,346.0,"Portray, display & decorate",False
4,https://collections.si.edu/search/detail/edanm...,SAAM,edanmdm:saam_2017.23.2,Elizabeth Waters (Mrs. Richard) Cromwell,"{""Credit Line"": ""Smithsonian American Art Muse...","{""Date"": ""ca. 1791-1794""}","{""Object number"": ""2017.23.2""}","{""Artist"": ""Charles Peale Polk, born Annapolis...",None,"{""Type"": ""Painting""}",...,0,None,None,[Paintings],[paintings],['paintings'],paintings,346.0,"Portray, display & decorate",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12658,https://collections.si.edu/search/detail/edanm...,NMAH,edanmdm:nmah_1000617,"2 Dollars, Maryland, 1775","{""Credit Line"": ""The Heirs of William and Denn...","{""date made"": ""1775"", ""date on object"": ""1775-...","{""ID Number"": ""NU.220760.018"", ""accession numb...","{""maker"": ""Green, Frederick""}","{""Location"": ""Currently not on view""}","{""Object Name"": ""currency\nnote""}",...,0,None,None,"[Money, note]","[money, note]","['money', 'note']",bank notes,14.0,Pay & exchange,False
12659,https://collections.si.edu/search/detail/edanm...,NMAH,edanmdm:nmah_1000616,"2 2/3 Dollars, Maryland, 1775","{""Credit Line"": ""The Heirs of William and Denn...","{""date made"": ""1775"", ""date on object"": ""1775-...","{""ID Number"": ""NU.220760.017"", ""accession numb...","{""maker"": ""Green, Frederick""}","{""Location"": ""Currently not on view""}","{""Object Name"": ""currency\nnote""}",...,0,None,None,"[Money, note]","[money, note]","['money', 'note']",bank notes,14.0,Pay & exchange,False
12660,https://collections.si.edu/search/detail/edanm...,NMAH,edanmdm:nmah_1000614,Washington City Canal Lottery Ticket,"{""Credit Line"": ""Gift of The Heirs of William ...","{""date made"": ""ca 1796""}","{""ID Number"": ""DL.220760.016"", ""

In [25]:
# comparing the auto-mapped vs. manually-mapped CSVs to document every change made by hand,
# and exporting those differences as 'override' tables that can be reapplied later


# 1) input / output paths

# input
auto_file = Path("final_mapped_database.csv")
manual_file = Path("final_mapped_database_manual.csv")

# output
overrides_canonical_file = Path("manual_overrides_canonical.csv")
overrides_action_file = Path("manual_overrides_action.csv")
audit_rowlevel_file = Path("manual_overrides_rowlevel.csv")

# 2) read both CSVs
auto_df = pd.read_csv(auto_file)
manual_df = pd.read_csv(manual_file)

# 3) fixed join key and other important columns
id_col = "EDANurl"  # Smithsonian’s unique record URL
group_col = "object_types_str"  # original object-type label
fields_to_compare = ["canonical_type", "action_family"]

# 4) merge so each row shows auto + manual side by side
merged = auto_df[[id_col, group_col] + fields_to_compare] \
           .merge(
               manual_df[[id_col, group_col] + fields_to_compare],
               on=[id_col, group_col],
               suffixes=("_auto", "_manual"),
               how="inner"
           )

# 5) detect changed rows
changed_mask = False
for f in fields_to_compare:
    changed = merged[f"{f}_auto"].astype(str).fillna("") != \
              merged[f"{f}_manual"].astype(str).fillna("")
    changed_mask = changed if isinstance(changed_mask, bool) else (changed_mask | changed)

changed_rows = merged[changed_mask].copy()
changed_rows.to_csv(audit_rowlevel_file, index=False)

# 6) build one-to-one overrides by original label
def build_override(field):
    a, m = f"{field}_auto", f"{field}_manual"
    g = merged.groupby(group_col)[[a, m]] \
              .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else s.iloc[0]) \
              .reset_index().rename(columns={a: "auto", m: "manual"})
    return g[g["auto"].astype(str).fillna("") != g["manual"].astype(str).fillna("")]

ov_canonical = build_override("canonical_type")
ov_action    = build_override("action_family")

if not ov_canonical.empty:
    ov_canonical.rename(columns={group_col: "original_label",
                                 "manual": "canonical_type_manual"}, inplace=True)
    ov_canonical[["original_label", "canonical_type_manual"]] \
        .to_csv(overrides_canonical_file, index=False)

if not ov_action.empty:
    ov_action.rename(
        columns={
            group_col: "object_type",       # keep object type for reference
            "auto": "previous_action",   # old action
            "manual": "new_action"         # new action
        },
        inplace=True
    )
    ov_action.to_csv(overrides_action_file, index=False)

In [26]:
# before creating the JSON structure from the final database to be used in the D3 treemap,
# I want to standardize the canonical_type values a bit more, since there are some categories that are very similar and will
# clutter the visualization. I will do this by merging some of the categories into broader ones, following these criteria:
# 1. create a table with all the unique canonical_type values and some other columns that will help decide how to group them)
# 2. use a mix of LLM prompting and manual review to define broader categories
# 3. create a mapping dictionary from the table and apply it to the final database

# This further canonization is done for display purposes.
# At this stage the action_family column has already been assigned using the
# original, more specific object types (e.g. "tinderboxes", "ballot boxes").
# Changing the canonical_type now will not affect the verbs/actions logic;
# it only merges similar objects (e.g. all bowls → "bowls") so the treemap
# labels are cleaner and easier to read.

# grouping by unique canonical_type and keeping one representative row for each type by adding some additional columns with details
unique_canon_df = ( 
    df.groupby("canonical_type", as_index=False)
    .first()[["canonical_type", "object_types_list", "objectType", "physicalDescription"]] 
    )

# saving the file to review and further canonization, where possible, with the support of an LLM
unique_canon_df.to_csv(Path("unique_canonical_types_for_review.csv"), index=False)

In [27]:
# loading the necessary CSV files
df_with_action = pd.read_csv("final_mapped_database_manual.csv")
canon = pd.read_csv("unique_canonical_types_canonized.csv")

# build a mapping {original -> suggested}
mapping = dict(zip(canon["canonical_type"], canon["suggested_canonical"]))

df_canonized = df_with_action.copy()
df_canonized["canonical_type"] = (
    df_canonized["canonical_type"].map(mapping).fillna(df_canonized["canonical_type"])
)

df_canonized.to_csv("final_mapped_database_canonized.csv", index=False)

# final corrections
df_with_action_canonized = pd.read_csv("final_mapped_database_canonized_manualfix.csv")

In [28]:
# replicating the same process as above to document the final manual fixes
# comparing the original vs manually-fixed CSVs to capture every change made by hand,
# and exporting those differences as "override" tables


# 1) input / output paths

# input
auto_file = Path("final_mapped_database_canonized.csv") # before final manual fixes
manual_file = Path("final_mapped_database_canonized_manualfix.csv") # after final manual fixes

# output
overrides_canonical_file = Path("manual_overrides_canonical_final.csv")
overrides_action_file = Path("manual_overrides_action_final.csv")
audit_rowlevel_file = Path("manual_overrides_rowlevel_final.csv")

# 2) read both CSVs
auto_df = pd.read_csv(auto_file)
manual_df = pd.read_csv(manual_file)

# 3) fixed join key and other important columns
id_col = "EDANurl"
group_col = "object_types_str"
fields_to_compare = ["canonical_type", "action_family"]

# 4) merge so each row shows auto + manual side by side
merged = auto_df[[id_col, group_col] + fields_to_compare] \
           .merge(
               manual_df[[id_col, group_col] + fields_to_compare],
               on=[id_col, group_col],
               suffixes=("_auto", "_manual"),
               how="inner"
           )

# 5) detect changed rows
changed_mask = False
for f in fields_to_compare:
    changed = merged[f"{f}_auto"].astype(str).fillna("") != \
              merged[f"{f}_manual"].astype(str).fillna("")
    changed_mask = changed if isinstance(changed_mask, bool) else (changed_mask | changed)

changed_rows = merged[changed_mask].copy()
changed_rows.to_csv(audit_rowlevel_file, index=False)

# 6) build one-to-one overrides by original label
def build_override(field):
    a, m = f"{field}_auto", f"{field}_manual"
    g = merged.groupby(group_col)[[a, m]] \
              .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else s.iloc[0]) \
              .reset_index().rename(columns={a: "auto", m: "manual"})
    return g[g["auto"].astype(str).fillna("") != g["manual"].astype(str).fillna("")]

ov_canonical = build_override("canonical_type")
ov_action    = build_override("action_family")

if not ov_canonical.empty:
    ov_canonical.rename(
        columns={
            group_col: "original_label",
            "manual": "canonical_type_manual"
        },
        inplace=True
    )
    ov_canonical[["original_label", "canonical_type_manual"]] \
        .to_csv(overrides_canonical_file, index=False)

if not ov_action.empty:
    ov_action.rename(
        columns={
            group_col: "object_type", # keep object type for reference
            "auto": "previous_action", # old action
            "manual": "new_action"     # new action
        },
        inplace=True
    )
    ov_action.to_csv(overrides_action_file, index=False)


In [29]:
# creating the nested JSON structure for a D3 treemap

df_with_action_canonized = pd.read_csv("final_mapped_database_canonized_manualfix.csv")

# keeping only what is needed from the final database
base = (
    df_with_action_canonized[["action_family", "canonical_type"]]
)

# aggregating counts: one row per (action_family, canonical_type) with a count
#    - .size() counts how many records fall into each pair of keys
#    - renamig that "size" column to "count" for clarity
counts = (
    base
    .groupby(["action_family", "canonical_type"], as_index=False)
    .size()
    .rename(columns={"size": "count"})
)

# sorting action families in alphabetical order, then object types by descending count
counts = counts.sort_values(
    by=["action_family", "count", "canonical_type"],
    ascending=[True, False, True],
)

# building the nested structure that the D3 treemap expects:
#    {
#      "name": "root",
#      "children": [
#        {
#          "name": "<action_family>",
#          "children": [
#            { "name": "<canonical_type>", "value": <count> },
#            ...
#          ]
#        },
#        ...
#      ]
#    }

# the root is the top-level folder that contains everything (the database)
# each action_family is a child of the root
# each canonical_type is a child of the action_family
# each canonical_type has a value, which is its count (the treemap’s rectangles are sized by these values)

data = {"name": "root", "children": []}

# grouping the counts by action_family
for action_family, group_df in counts.groupby("action_family"):
    # for each canonical_type inside this action_family, make a leaf node
    children = [
        {
            "name": row["canonical_type"],
            "value": int(row["count"])
        }
        for _, row in group_df.iterrows()
    ]

    # adding this action_family (internal node) to the root’s children
    data["children"].append({
        "name": action_family,
        "children": children
    })

# saving the dictionary as a JSON file
# creating a path pointing to the output file
out_path = Path("treemap_data.json")
# converting the Python dictionary into a JSON string
out_path.write_text(
    json.dumps(data, ensure_ascii=False, indent=2), 
    encoding="utf-8"
    )


54601

In [30]:
# creating another JSON structure for the 3rd click level that maps each object type (column canonical_type)
# with its list of individual records (title, unitCode, collectionsURL)

#  this will be its shape:

#   {
#     "chair": [
#       {"title": "...", "unitCode": "...", "collectionsURL": "..."},
#       {"title": "...", "unitCode": "...", "collectionsURL": "..."}
#     ],
#     "spoon": [
#       {"title": "...", "unitCode": "...", "collectionsURL": "..."}
#     ]
#   }


# choosing the columns that will be used
col_object_type = "canonical_type" # this will be the grouping key
col_title = "title"           
col_unit = "unitCode"    
col_url = "collectionsURL"
col_identifier = "EDANurl"
col_action = "action_family" # this column will be used to filter the details panel not only by object type but also by action family

# preparing an empty dictionary to fill as the function iterates over rows
lookup = {}

# loop over every row in df_with_action
for _, row in df_with_action_canonized.iterrows():
    # get the object type for this row
    obj_type = row[col_object_type]
    # if the object type column is empty, skip it (just in case)
    if not obj_type:
        continue
    # create the small record we want to store
    item = {
        "title": row[col_title],
        "unitCode": row[col_unit],
        "collectionsURL": row[col_url],
        "EDANurl": row[col_identifier],
        "action_family": row[col_action],
    }
    # if the object type on the current row hasn’t appeared yet, create an empty list for it
    # this is to populate the lookup dictionary with every object type once (the first time it appears)     
    if obj_type not in lookup:
        lookup[obj_type] = []
    # and then append the record on the current row to the list for the current object type
    lookup[obj_type].append(item)

# saving the dictionary as a JSON file
# creating a path pointing to the output file
out_path = Path("object_details.json")
# converting the Python dictionary into a JSON string
out_path.write_text(
    json.dumps(lookup, ensure_ascii=False, indent=2), 
    encoding="utf-8"
    )




3308034